In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Abrimos todos los csv, notese que formateamos los datos de las fechas para acomodarlos al formato usado en varias funciones de pandas (que usaremos mas adelante)
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
train = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
item_categories = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')
sales = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv',  parse_dates=['date'], infer_datetime_format=True, dayfirst=True)
sales2 = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv',  parse_dates=['date'], infer_datetime_format=True, dayfirst=True)

#
# juntamos los data set para tener solo 2 data sets, uno de ellos lo adaptaremos para usarlo de test y el otro tendra el resto de datos
df_item = pd.merge(items,item_categories,on='item_category_id',how='inner')
sales_t = pd.merge(sales,shops,on='shop_id',how='inner')
sales = pd.merge(sales_t,df_item,on='item_id',how='inner')

df_item2 = pd.merge(items,item_categories,on='item_category_id',how='inner')
sales_t2 = pd.merge(sales2,shops,on='shop_id',how='inner')
sales2 = pd.merge(sales_t2,df_item2,on='item_id',how='inner')

#
# Separamos los datos que nos piden que usemos como test en "sales"
sales = sales[sales['shop_id'].isin(test['shop_id'].unique())]
sales = sales[sales['item_id'].isin(test['item_id'].unique())]

#
# Eliminamos los datos atipicos para aumentar la precision del modelo
sales = sales[(sales.item_price < 300000 )& (sales.item_cnt_day < 1000)]
sales = sales[sales.item_price > 0].reset_index(drop = True)
sales2 = sales2[(sales2.item_price < 300000 )& (sales2.item_cnt_day < 1000)]
sales2 = sales2[sales2.item_price > 0].reset_index(drop = True)
sales_recien_cargado = sales

#
# Los datos estan divididos en bloques de fechas, de manera que agrupar los datos para poder usar las ventas de fechas anteriores como datos de entrenamiento
# Con esta agrupacion podriamos usar la media del precio que tiene un item a lo largo del tiempo o en nuestro caso el numero de ventas que ha ido teniendo a lo largo del tiempo
# Esto lo ahcemos tanto con los datos de entrenamiento como los de test ya que deben tener el mismo formato para pasarlos por el mismo modelo
sales = sales.groupby(["date_block_num","shop_id","item_id"])[['date_block_num','date', 'shop_id', 'item_id', 'item_price', 'item_cnt_day']].agg({"date_block_num":'mean',"date":["min",'max'],"item_price":"mean","item_cnt_day":"sum"})
sales = sales.item_cnt_day.apply(list).reset_index()
sales_data = pd.merge(test,sales,on = ['item_id','shop_id'],how = 'left')
sales2 = sales2.groupby(["date_block_num","shop_id","item_id"])[['date_block_num','date', 'shop_id', 'item_id', 'item_price', 'item_cnt_day']].agg({"date_block_num":'mean',"date":["min",'max'],"item_price":"mean","item_cnt_day":"sum"})
sales2 = sales2.item_cnt_day.apply(list).reset_index()
sales_data2 = sales2
sales_prueba = sales_data


# Finalmente reorganizamos las tablas para que tengan un formato legible por el modelo de regresion lineal, dejandolo de manera que:
# Cada fila es una iteracion del algoritmo y cada columna un valor de X dentro de la ecuacion de la regresion lineal.
# siendo las filas distintos objetos y las columnas las ventas a lo largo del tiempo. 
# Cada columna es un bloque de tiempo que simboliza un mes en concreto de manera ascendiente siendo 0 el mes mas antiguo junio del 2013 y 33 el mes mas reciente cotubre del 2015
# Como pretendemos predecir VENTAS usando las ventas pasadas, usaremos la ultima columna como Y, valor objetivo de la prediccion
sales_data.fillna(0,inplace = True)
sales_data.drop(['shop_id','item_id'],inplace = True, axis = 1)
sales_data = sales_data.pivot_table(index = 'ID', columns='date_block_num', values = 'sum', aggfunc='sum')
sales_data = sales_data.fillna(0)
sales_data2.fillna(0,inplace = True)
sales_data2 = sales_data2.pivot_table(index = ['shop_id','item_id'], columns='date_block_num', values = 'sum', aggfunc='sum')
sales_data2 = sales_data2.fillna(0)


# Preparamos los datos de entrenamiento y test; reservando la ultima columna (representando al ultimo mes otorgado por el dataset) como dato objetivo y dejaremos el resto como datos de entrenamiento (basaremos el modelo en las ventas de los meses anteriores)
# Los datos han sido previamente separados en dos paquetes: uno para entrenamiento y otro para test. Hecho de esta manera especificamente ya qeu el problema nos pide que usemos un paquete de test en espeficico para ahcer las predicciones finales.
x_train = sales_data2[sales_data2.columns[:-1]]
y_train = sales_data2[sales_data2.columns[-1]]
x_test = sales_data[sales_data.columns[:-1]]
y_test = sales_data[sales_data.columns[-1]]


# Creamos el modelo usando sklearn
model = LinearRegression()
model.fit(x_train, y_train)

# Obtenemos nuestras predicciones segun nuestro modelo
y_pred = model.predict(x_test)

# Echamos un pequeño vistazo al rmse para ver cuanto nso desviamos del resultado real
print('RMSE: ', mean_squared_error(y_test, y_pred, squared=False))

# Creamos el fichero csv de evaluacion con las predicciones obtenidas
submission = pd.DataFrame({'ID':x_test.index,'item_cnt_month':y_pred})
submission.to_csv('submission.csv',index = False)
pd.read_csv('submission.csv')

In [ ]:
# Datos recien cargados.
sales_recien_cargado.head()

In [ ]:
# Datos despues de podar los datos no utiles y realizar la agrupacion de ventas.
sales.head()

In [ ]:
# Datos adaptados a las condiciones del problema.
sales_prueba.head()

In [ ]:
# Datos finalmente formateados para ser utilizados por el modelo.
sales_data.head()